# 03 · Correlation & Cohort Analysis
> Day 7 — correlation matrix, churn‑tenure, churn‑MRR cohorts

## 1. Setup & Data Load

In [8]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
_CWD = Path.cwd()
if (_CWD / "data").exists():               # we're already at project root
    PROJECT_ROOT = _CWD
else:                                      # we’re inside notebooks/
    PROJECT_ROOT = _CWD.parent
    
CLEAN_PATH   = PROJECT_ROOT / "data" / "clean" / "telco_clean.parquet"
FIG_DIR      = PROJECT_ROOT / "figures"
FIG_DIR.mkdir(exist_ok=True)

df = pd.read_parquet(CLEAN_PATH)
print(df.shape)


(7043, 21)


## 2. Correlation Matrix Heatmap

In [4]:
# --- numeric correlation matrix ---
num_cols = df.select_dtypes(include="number").columns
corr = df[num_cols].corr(method="pearson").round(2)

plt.figure(figsize=(10, 8))
sns.heatmap(
    corr,
    annot=True, fmt=".2f",
    cmap="RdBu_r",
    vmin=-1, vmax=1,
    linewidths=.5, square=True
)
plt.title("Pearson Correlation Matrix (Numeric Features)")
for ext in ("png", "svg"):
    plt.savefig(FIG_DIR / f"corr_heatmap.{ext}", bbox_inches="tight")
plt.close()

## 3. Churn × Tenure Cohorts

In [15]:
# Bucket tenure into 6‑month bins
df["tenure_bin"] = pd.cut(df["tenure"], bins=range(0, 72+6, 6), right=False)

cohorts = (
    df.groupby("tenure_bin", observed=True)["Churn"]
      .value_counts(normalize=True)
      .rename("pct")
      .reset_index()
      .query("Churn == 'Yes'")
)

plt.figure(figsize=(10, 4))
sns.barplot(data=cohorts, x="tenure_bin", y="pct", color="#3b8eea")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Churn Rate")
plt.xlabel("Tenure (months, 6‑month bins)")
plt.title("Churn Rate vs Tenure")
for ext in ("png", "svg"):
    plt.savefig(FIG_DIR / f"churn_vs_tenure.{ext}", bbox_inches="tight")
plt.close()

## 4. Churn × MonthlyCharges Cohorts

In [19]:
# Quartile‑based buckets
df["mrr_bin"] = pd.qcut(df["MonthlyCharges"], q=4, labels=["Q1","Q2","Q3","Q4"])

mrr_cohort = (
    df.groupby("mrr_bin", observed=True)["Churn"]
      .value_counts(normalize=True)
      .rename("pct")
      .reset_index()
      .query("Churn == 'Yes'")
      .sort_values("mrr_bin")
)

plt.figure(figsize=(6, 4))
sns.barplot(data=mrr_cohort, x="mrr_bin", y="pct", color="#3b8eea")
plt.ylabel("Churn Rate")
plt.xlabel("MonthlyCharges quartile")
plt.title("Churn Rate vs MonthlyCharges")
for ext in ("png", "svg"):
    plt.savefig(FIG_DIR / f"churn_vs_mrr.{ext}", bbox_inches="tight")
plt.close()

## 5. Quick Findings

### 1 . Correlation matrix (Pearson)

| Pair | r value | Insight |
|------|---------|---------|
| **tenure ↔ TotalCharges** | **0.82** | TotalCharges is essentially tenure × rate; they carry very similar information.  In tree models they’ll be redundant. |
| **MonthlyCharges ↔ TotalCharges** | **0.65** | As expected: customers who pay more per month also accumulate higher lifetime spend. |
| **tenure ↔ MonthlyCharges** | **0.25** | Weak positive correlation – higher‑paying plans are *slightly* more common among long‑tenure customers. No multicollinearity issue here. |

### 2 . Churn rate vs Tenure (6‑month cohorts)

| Tenure bin (months) | Churn Rate |
|---------------------|------------|
| **[0, 6)** (early life) | **≈ 0.54** |
| [6, 12) | ≈ 0.37 |
| [12, 18) | ≈ 0.34 |
| [18, 24) | ≈ 0.25 |
| [24, 30) | ≈ 0.23 |
| [30, 42) | ≈ 0.22 |
| [42, 48) | ≈ 0.18 |
| [48, 60) | ≈ 0.14–0.15 |
| [60, 72) | ≈ 0.08–0.09 |

**Take‑away:** More than half of customers churn within the first 6 months; rate declines steadily thereafter. Focus retention spend on the first year (“on‑boarding” + early‑life incentives).

### 3 . Churn rate vs MonthlyCharges (quartiles)

| MonthlyCharges quartile | Approx. range (USD) | Churn Rate |
|-------------------------|---------------------|------------|
| **Q1** (lowest) | ≤ \$35 | **≈ 0.11** |
| **Q2** | \$35 – \$55 | ≈ 0.25 |
| **Q3** | \$55 – \$75 | **≈ 0.37 (highest)** |
| **Q4** (highest) | ≥ \$75 | ≈ 0.32 |

**Take‑away:** Mid‑to‑upper‑pricing tiers (Q3/Q4) show the highest churn. Low‑price customers (Q1) are comparatively sticky. Actionables:  
* audit whether Q3/Q4 customers perceive sufficient value;  
* consider loyalty perks or contract discounts for high‑MRR segments.

---

**Meta:** Dataset = 7 043 rows, 21 columns (post‑clean). All calculations use `observed=True` in `groupby` to ignore empty bins.
